# Exercise I

In [16]:
#standard imports
import pandas as pd
import numpy as np

### Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url connection string formatted like in the example at the start of this lesson.

In [17]:
import env

In [106]:
def get_db_url(db):
    from env import host, user, password
    url = f'mysql+pymysql://{user}:{password}@{host}/{db}'
    return url

### Use your function to obtain a connection to the employees database.

In [21]:
url = get_db_url('employees')

In [20]:
df = pd.read_sql('select * from employees limit 5', url)
df

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


### Once you have successfully run a query:

- a. Intentionally make a typo in the database url. What kind of error message do you see?

- b. Intentionally make an error in your SQL query. What does the error message look like?

### Read the employees and titles tables into two separate DataFrames.

In [22]:
df1 = pd.read_sql('select * from employees limit 5', url)
df1

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


In [23]:
df2 = pd.read_sql('select * from titles limit 5', url)
df2

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01


### How many rows and columns do you have in each DataFrame? Is that what you expected?

> i limited to 5 rows so colums and rows displayed as expected

### Display the summary statistics for each DataFrame.

In [24]:
pd.read_sql('describe employees;', url)

,Field,Type,Null,Key,Default,Extra
0,emp_no,int,NO,PRI,None,
1,birth_date,date,NO,,None,
2,first_name,varchar(14),NO,,None,
3,last_name,varchar(16),NO,,None,
4,gender,"enum('M','F')",NO,,None,
5,hire_date,date,NO,,None,


In [25]:
pd.read_sql('describe titles;', url)

,Field,Type,Null,Key,Default,Extra
0,emp_no,int,NO,PRI,None,
1,title,varchar(50),NO,PRI,None,
2,from_date,date,NO,PRI,None,
3,to_date,date,YES,,None,


### How many unique titles are in the titles DataFrame?

In [26]:
pd.read_sql('''
        select title
        from titles
        group by title
        ''', url
        )

,title
0,Senior Engineer
1,Staff
2,Engineer
3,Senior Staff
4,Assistant Engineer
5,Technique Leader
6,Manager


### What is the oldest date in the to_date column?

In [27]:
pd.read_sql('''
        select to_date
        from titles
        order by to_date  
        limit 1
        ''', url
        )

,to_date
0,1985-03-01


### What is the most recent date in the to_date column?

In [29]:
query = '''
select to_date
from titles
where to_date not like '9999%%'
order by to_date desc
limit 1
;
'''

In [30]:
pd.read_sql(query, url)

,to_date
0,2002-08-01


# Exercise II

### Copy the users and roles DataFrames from the examples above.

In [32]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})

# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})

In [33]:
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


In [34]:
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


### What is the result of using a right join on the DataFrames?

In [35]:
pd.merge(users, roles, how='right', left_on='role_id', right_on='id')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1,admin
1,2.0,joe,2.0,2,author
2,3.0,sally,3.0,3,reviewer
3,4.0,adam,3.0,3,reviewer
4,NaN,NaN,NaN,4,commenter


### What is the result of using an outer join on the DataFrames?

In [36]:
pd.merge(users, roles, how='outer', left_on='role_id', right_on='id')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1.0,admin
1,2.0,joe,2.0,2.0,author
2,3.0,sally,3.0,3.0,reviewer
3,4.0,adam,3.0,3.0,reviewer
4,5.0,jane,NaN,NaN,NaN
5,6.0,mike,NaN,NaN,NaN
6,NaN,NaN,NaN,4.0,commenter


### What happens if you drop the foreign keys from the DataFrames and try to merge them?

In [38]:
pd.merge(users, roles, how='inner')

,id,name,role_id


### Load the mpg dataset from PyDataset.

In [42]:
from pydataset import data

### Output and read the documentation for the mpg dataset.

In [46]:
mpg = data('mpg')

In [47]:
data('mpg', show_doc = True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




### How many rows and columns are in the dataset?

In [48]:
mpg.shape

(234, 11)

### Check out your column names and perform any cleanup you may want on them.

In [50]:
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


### Display the summary statistics for the dataset.

In [52]:
mpg.describe().round(1)

,displ,year,cyl,cty,hwy
count,234.0,234.0,234.0,234.0,234.0
mean,3.5,2003.5,5.9,16.9,23.4
std,1.3,4.5,1.6,4.3,6.0
min,1.6,1999.0,4.0,9.0,12.0
25%,2.4,1999.0,4.0,14.0,18.0
50%,3.3,2003.5,6.0,17.0,24.0
75%,4.6,2008.0,8.0,19.0,27.0
max,7.0,2008.0,8.0,35.0,44.0


### How many different manufacturers are there?

In [67]:
mpg.groupby('manufacturer').describe().round(1)

displ                                     year          ...  \
             count mean  std  min  25%  50%  75%  max count    mean  ...   
manufacturer                                                         ...   
audi          18.0  2.5  0.7  1.8  2.0  2.8  3.0  4.2  18.0  2003.5  ...   
chevrolet     19.0  5.1  1.4  2.4  4.4  5.3  5.8  7.0  19.0  2004.7  ...   
dodge         37.0  4.4  0.9  2.4  3.8  4.7  4.7  5.9  37.0  2004.1  ...   
ford          25.0  4.5  0.5  3.8  4.0  4.6  4.6  5.4  25.0  2002.6  ...   
honda          9.0  1.7  0.1  1.6  1.6  1.6  1.8  2.0   9.0  2003.0  ...   
hyundai       14.0  2.4  0.4  2.0  2.1  2.4  2.7  3.3  14.0  2004.1  ...   
jeep           8.0  4.6  1.0  3.0  3.9  4.7  5.0  6.1   8.0  2005.8  ...   
land rover     4.0  4.3  0.3  4.0  4.2  4.3  4.4  4.6   4.0  2003.5  ...   
lincoln        3.0  5.4  0.0  5.4  5.4  5.4  5.4  5.4   3.0  2002.0  ...   
mercury        4.0  4.4  0.5  4.0  4.0  4.3  4.7  5.0   4.0  2003.5  ...   
nissan        13.0  3.3  0.9  2.4  2.5  3.3  3.5  5.6  13.0  2003.8  ...   
pontiac        5.0  4.0  0.8  3.1  3.8  3.8  3.8  5.3   5.0  2002.6  ...   
subaru        14.0  2.5  0.1  2.2  2.5  2.5  2.5  2.5  14.0  2004.1  ...   
toyota        34.0  3.0  0.9  1.8  2.2  2.7  3.4  5.7  34.0  2002.7  ...   
volkswagen    27.0  2.3  0.4  1.8  2.0  2.0  2.5  3.6  27.0  2002.7  ...   

               cty         hwy                                           
               75%   max count  mean  std   min   25%   50%   75%   max  
manufacturer                                                             
audi          18.8  21.0  18.0  26.4  2.2  23.0  25.0  26.0  27.8  31.0  
chevrolet     16.5  22.0  19.0  21.9  5.1  14.0  17.0  23.0  26.0  30.0  
dodge         15.0  18.0  37.0  17.9  3.6  12.0  16.0  17.0  21.0  24.0  
ford          15.0  18.0  25.0  19.4  3.3  15.0  17.0  18.0  22.0  26.0  
honda         25.0  28.0   9.0  32.6  2.6  29.0  32.0  32.0  34.0  36.0  
hyundai       19.8  21.0  14.0  26.9  2.2  24.0  26.0  26.5  28.0  31.0  
jeep          15.0  17.0   8.0  17.6  3.2  12.0  16.2  18.5  19.2  22.0  
land rover    12.0  12.0   4.0  16.5  1.7  15.0  15.0  16.5  18.0  18.0  
lincoln       11.5  12.0   3.0  17.0  1.0  16.0  16.5  17.0  17.5  18.0  
mercury       13.2  14.0   4.0  18.0  1.2  17.0  17.0  18.0  19.0  19.0  
nissan        19.0  23.0  13.0  24.6  5.1  17.0  20.0  26.0  27.0  32.0  
pontiac       18.0  18.0   5.0  26.4  1.1  25.0  26.0  26.0  27.0  28.0  
subaru        20.0  21.0  14.0  25.6  1.2  23.0  25.0  26.0  26.0  27.0  
toyota        21.0  28.0  34.0  24.9  6.2  15.0  20.0  26.0  29.8  37.0  
volkswagen    21.0  35.0  27.0  29.2  5.3  23.0  26.0  29.0  29.0  44.0  

[15 rows x 40 columns]

### How many different models are there?

In [68]:
mpg.groupby('model').describe().round(1)

displ                                     year          \
                       count mean  std  min  25%  50%  75%  max count    mean   
model                                                                           
4runner 4wd              6.0  3.5  0.8  2.7  2.9  3.4  3.8  4.7   6.0  2002.0   
a4                       7.0  2.3  0.5  1.8  1.9  2.0  2.8  3.1   7.0  2002.9   
a4 quattro               8.0  2.4  0.6  1.8  2.0  2.4  2.9  3.1   8.0  2003.5   
a6 quattro               3.0  3.4  0.7  2.8  3.0  3.1  3.6  4.2   3.0  2005.0   
altima                   6.0  2.8  0.5  2.4  2.4  2.5  3.2  3.5   6.0  2005.0   
c1500 suburban 2wd       5.0  5.5  0.3  5.3  5.3  5.3  5.7  6.0   5.0  2006.2   
camry                    7.0  2.7  0.5  2.2  2.3  2.4  3.0  3.5   7.0  2002.9   
camry solara             7.0  2.6  0.4  2.2  2.3  2.4  3.0  3.3   7.0  2002.9   
caravan 2wd             11.0  3.4  0.5  2.4  3.3  3.3  3.8  4.0  11.0  2003.1   
civic                    9.0  1.7  0.1  1.6  1.6  1.6  1.8  2.0   9.0  2003.0   
corolla                  5.0  1.8  0.0  1.8  1.8  1.8  1.8  1.8   5.0  2002.6   
corvette                 5.0  6.2  0.5  5.7  5.7  6.2  6.2  7.0   5.0  2004.4   
dakota pickup 4wd        9.0  4.4  0.6  3.7  3.9  4.7  4.7  5.2   9.0  2004.0   
durango 4wd              7.0  5.0  0.7  3.9  4.7  4.7  5.4  5.9   7.0  2004.1   
expedition 2wd           3.0  5.1  0.5  4.6  5.0  5.4  5.4  5.4   3.0  2002.0   
explorer 4wd             6.0  4.3  0.4  4.0  4.0  4.0  4.4  5.0   6.0  2002.0   
f150 pickup 4wd          7.0  4.7  0.5  4.2  4.4  4.6  5.0  5.4   7.0  2001.6   
forester awd             6.0  2.5  0.0  2.5  2.5  2.5  2.5  2.5   6.0  2005.0   
grand cherokee 4wd       8.0  4.6  1.0  3.0  3.9  4.7  5.0  6.1   8.0  2005.8   
grand prix               5.0  4.0  0.8  3.1  3.8  3.8  3.8  5.3   5.0  2002.6   
gti                      5.0  2.2  0.4  2.0  2.0  2.0  2.0  2.8   5.0  2002.6   
impreza awd              8.0  2.4  0.1  2.2  2.4  2.5  2.5  2.5   8.0  2003.5   
jetta                    9.0  2.3  0.4  1.9  2.0  2.0  2.5  2.8   9.0  2003.0   
k1500 tahoe 4wd          4.0  5.7  0.6  5.3  5.3  5.5  5.9  6.5   4.0  2003.5   
land cruiser wagon 4wd   2.0  5.2  0.7  4.7  5.0  5.2  5.4  5.7   2.0  2003.5   
malibu                   5.0  3.0  0.6  2.4  2.4  3.1  3.5  3.6   5.0  2004.4   
maxima                   3.0  3.2  0.3  3.0  3.0  3.0  3.2  3.5   3.0  2002.0   
mountaineer 4wd          4.0  4.4  0.5  4.0  4.0  4.3  4.7  5.0   4.0  2003.5   
mustang                  9.0  4.4  0.5  3.8  4.0  4.6  4.6  5.4   9.0  2004.0   
navigator 2wd            3.0  5.4  0.0  5.4  5.4  5.4  5.4  5.4   3.0  2002.0   
new beetle               6.0  2.1  0.3  1.9  1.9  2.0  2.4  2.5   6.0  2002.0   
passat                   7.0  2.4  0.7  1.8  1.9  2.0  2.8  3.6   7.0  2002.9   
pathfinder 4wd           4.0  4.0  1.1  3.3  3.3  3.6  4.4  5.6   4.0  2003.5   
ram 1500 pickup 4wd     10.0  5.0  0.5  4.7  4.7  4.7  5.2  5.9  10.0  2005.3   
range rover              4.0  4.3  0.3  4.0  4.2  4.3  4.4  4.6   4.0  2003.5   
sonata                   7.0  2.6  0.3  2.4  2.4  2.4  2.5  3.3   7.0  2002.9   
tiburon                  7.0  2.3  0.4  2.0  2.0  2.0  2.7  2.7   7.0  2005.4   
toyota tacoma 4wd        7.0  3.3  0.6  2.7  2.7  3.4  3.7  4.0   7.0  2002.9   

                        ...   cty         hwy                               \
                        ...   75%   max count  mean  std   min   25%   50%   
model                   ...                                                  
4runner 4wd             ...  15.8  16.0   6.0  18.8  1.5  17.0  17.5  19.5   
a4                      ...  20.5  21.0   7.0  28.3  2.0  26.0  26.5  29.0   
a4 quattro              ...  18.2  20.0   8.0  25.8  1.2  25.0  25.0  25.0   
a6 quattro              ...  16.5  17.0   3.0  24.0  1.0  23.0  23.5  24.0   
altima                  ...  22.5  23.0   6.0  28.7  2.4  26.0  27.0  28.0   
c1500 suburban 2wd      ...  14.0  14.0   5.0  17.8  2.2  15.0  17.0  17.0   

### Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.

In [71]:
mpg['mileage_difference'] = mpg.hwy - mpg.cty
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mileage_difference
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10
...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8


### Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the city and highway mileage.

In [73]:
mpg['average_mileage'] = (mpg.cty + mpg.hwy) / 2

In [74]:
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mileage_difference,average_mileage
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0


### Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether the car has an automatic transmission.

In [80]:
mask = mpg.trans.str.startswith('auto')

In [92]:
mpg['is_automatic'] = mask
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mileage_difference,average_mileage,is_automotic,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5,True,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0,False,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5,False,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5,True,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5,True,True
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0,False,False
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0,True,True
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0,False,False


### Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?

In [132]:
mpg.groupby('manufacturer').average_mileage.mean().round(2).nlargest(1)

manufacturer
honda    28.5
Name: average_mileage, dtype: float64

### Do automatic or manual cars have better miles per gallon?

In [133]:
mpg.groupby('is_automatic').mean().average_mileage.round(2)

is_automatic
False    22.23
True     19.13
Name: average_mileage, dtype: float64

# Exercise III

### Use your get_db_url function to help you explore the data from the chipotle database.

In [102]:
import env

In [174]:
url = get_db_url('chipotle')

### What is the total price for each order?

In [111]:
query = '''
select order_id, ROUND(SUM(REPLACE(item_price,'$','')),2) as tot_purchase
from orders
group by order_id
;
'''

In [112]:
df = pd.read_sql(query, url)
df

,order_id,tot_purchase
0,1,11.56
1,2,16.98
2,3,12.67
3,4,21.00
4,5,13.70
...,...,...
1829,1830,23.00
1830,1831,12.90
1831,1832,13.20
1832,1833,23.50


In [ ]:
df = pd.read_sql(query, url)

### What are the most popular 3 items?

In [176]:
df = pd.read_sql('select * from orders', url)

In [177]:
df.groupby('item_name').quantity.sum().sort_values(ascending=False).head(3)

item_name
Chicken Bowl           761
Chicken Burrito        591
Chips and Guacamole    506
Name: quantity, dtype: int64

### Which item has produced the most revenue?

In [119]:
query = '''
select item_name, ROUND(SUM(REPLACE(item_price,'$','')),2)
from orders
group by item_name
order by ROUND(SUM(REPLACE(item_price,'$','')),2) desc
limit 1;
'''

In [120]:
df = pd.read_sql(query, url)
df

,item_name,"ROUND(SUM(REPLACE(item_price,'$','')),2)"
0,Chicken Bowl,7342.73


### Join the employees and titles DataFrames together.

In [136]:
url = get_db_url('employees')

In [142]:
employees_df = pd.read_sql('select * from employees', url)

In [143]:
titles_df = pd.read_sql('select * from titles', url)

In [145]:
emp_titles = pd.merge(employees_df, titles_df, how='outer', on='emp_no')
emp_titles.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01


### For each title, find the hire date of the employee that was hired most recently with that title.

In [179]:
emp_titles.groupby('title').hire_date.max()

title
Assistant Engineer    1999-12-24
Engineer              2000-01-28
Manager               1992-02-05
Senior Engineer       2000-01-01
Senior Staff          2000-01-13
Staff                 2000-01-12
Technique Leader      1999-12-31
Name: hire_date, dtype: object

In [184]:
emp_titles.pivot_table(index='title', values='hire_date', aggfunc='max')

,hire_date
title,
Assistant Engineer,1999-12-24
Engineer,2000-01-28
Manager,1992-02-05
Senior Engineer,2000-01-01
Senior Staff,2000-01-13
Staff,2000-01-12
Technique Leader,1999-12-31


### Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)

In [167]:
dept_df = pd.read_sql('''
                        select dept_no, title
                        from dept_emp
                        join titles
                        using(emp_no)
                        ''', url)

In [185]:
pd.crosstab(dept_df.title, dept_df.dept_no)

dept_no,d001,d002,d003,d004,d005,d006,d007,d008,d009
title,,,,,,,,,
Assistant Engineer,0,0,0,6445,7769,1831,0,378,298
Engineer,0,0,0,49649,58135,13852,0,2986,2362
Manager,2,2,2,4,2,4,2,2,4
Senior Engineer,0,0,0,42205,49326,11864,0,2570,2027
Senior Staff,13940,12139,12274,1270,1247,0,36191,11637,13925
Staff,16196,13929,14342,1478,1424,0,41808,13495,16150
Technique Leader,0,0,0,6557,7683,1795,0,393,309
